# `1.7 Radio Frequency 📡`

## <span style="color:tomato"> Overview </span>

The SX1272 is a transceiver integrated circuit (chip) that takes digital messages from the microcontroller and is capable of modulating them to a particular communication protocol such that they can be transmited and received as data packets with the antenna on the LunaSat. 

---

## <span style="color:tomato"> RF's Role In the Mission </span>

Radio Frequency (RF) is necessary not only to send data to the lander, but also to make sure that all LunaSats can maintain accurate timestamps for qualifying data via clock synchronization algorithms. You will discover that the RF system is the most power-intensive component on our LunaSat later in this notebook. To address this, we anticipate switching between two "modes" on the LunaSat on the Moon - one where data collection is occuring and sensors are active, and another where only necessary components and the RF components are active. Essentially, RF will never be active as long as sensors are active, and vice versa.

#### <span style="color:orange">Example Configuration Timeline</span>

#### | <span style="color:orange">**Temperature + Accelerometer On, RF Off**</span> (10 seconds) | <span style="color:orange">**RF On, All Sensors Off**</span> (5 seconds) | <span style="color:orange">**Temperature On, RF Off**</span> (7 seconds) |

In reality, each LunaSat will be assigned a transmit / receive window during which RF is on and sensors are off. This means that your final choice of configurations will not actually include any RF configurations. Additionally, RF parameters will be optimized on our end so as to ensure the success of network operations.

This notebook will ultimately show you GLEE's process of determining how to configure the RF system so that its power usage is minimized.

---

## <span style="color:tomato"> Configurations </span>

#### The SX1272 chip is designed to work with a particular communications protocol called LoRa™. There are several LoRa™ operating modes to choose from. Don't worry if you don't understand the jargon used below. We will highlight which modes are important later. 
- <span style="color:orange">**SLEEP**</span>: A low-power mode where only SPI and configuration registers are active and accessible.
    - Typically uses 0.1 µA and up to a max of 1 µA.
- <span style="color:orange">**STANDBY**</span>: The crystal oscillator and LoRa™ baseband blocks are turned on, while the RF front-end and PLLs are disabled.
    - Typically uses 1.4 mA and up to 1.6 mA
- <span style="color:orange">**IDLE**</span>
    - Typically uses 1.5 µA.
- <span style="color:orange">**FSTX**</span>: A frequency-synthesis mode for transmission, where the PLL selected for transmission is locked and active at the transmit frequency while the RF front-end is off
    - Typically uses 4.5 mA.
- <span style="color:orange">**FSRX**</span>: A frequency-synthesis mode for receiving, where the PLL selected for reception is locked and active at the receive frequency while the RF front-end is off
    - Typically uses 4.5 mA.
- <span style="color:orange">**TX**</span>: All remaining blocks required for transmitting are powered, the PA is ramped and the packet is sent. The SX1272 then goes into STANDBY mode.
    - Typically uses 90 mA at +17dBm with an external impedance transformation. 
- <span style="color:orange">**RXCONTINUOUS**</span>: All remaining blocks required for receiving are powered and data that is received is processed. Done continuously until the mode is changed.
    - Typically uses 11.6 mA with LnaBoost on and with a bandwidth of 250 kHz.
- <span style="color:orange">**RXSINGLE**</span>: All remaining blocks required for receiving are powered and data that is received is processed. Once a packet is received, the SX1272 goes into STANDBY mode.
    - Typically uses 11.6 mA with LnaBoost on and with a bandwidth of 250 kHz.
- <span style="color:orange">**CAD**</span>: The device will check a given channel to detect a LoRa preamble signal.

---

## <span style="color:tomato"> Creating Configurations </span>

- <span style="color:orange">**Modes**</span>: {"SLEEP", "STANDBY", "TX", "RXCONTINUOUS", "RXSINGLE", "IDLE", "FSTX", "FSRX", "CAD"}

#### There are a number of LoRa™ parameters we can also configure the SX1272 to have. You may recognize that some of these parameters were utilized in Module 9.

- <span style="color:orange">**Frequency**</span>: {Any value between 860 and 1000 (1 GHz)}
    - Frequency of radio waves received or transmitted by RF Module. Typically 915-928MHz corresponding to LoRas operation on unlicensed ISM Radio Band in US. Set to **915 MHz** in our code.
- <span style="color:orange">**Output Power**</span>: {7, 13, 17, 20}
    - Power of transceiver transmission. Set to **13 dBm** in our code. The higher the output power, the farther a signal will be able to travel before becoming undetectable by a receiver.
- <span style="color:orange">**Bandwidth**</span>: {125,250,500}
    - Bandwidth is the frequency range of chirps used to transmit data. Typically 250kHz for terrestrial usage in the US. Set to **125 kHz** in our code.
- <span style="color:orange">**LNA Boost**</span>: {"ON", "OFF"}
    - The Low-Noise Amplifier Boost or LNA is responsible for amplifying weak incoming signals from the antenna before they are processed by the receiver. By default it is set to **"OFF"** in our code.
- <span style="color:orange">**Spreading Factor**</span>: {7, 8, 9, 10, 11, 12}
    - The Spreading Factor dictates the number of chirps used to send a signal. Set to **12** in our code.
- <span style="color:orange">**Coding Rate**</span>: {5, 6, 7, 8}
    - Coding Rate effects data encoding redundancies. CR value is proportional to interference tolerance. Set to **6** in our code. Denoted as 4/#. (Higher - Able to handle more interference)
- <span style="color:orange">**Payload Length**</span>: {0-232}
    - Payload length is length of the data or information that is being transmitted or received excluding any additional overhead or control information. Payload length is expressed in bytes.

Certain configurations do not utilize all available settings. Below are the configuration formats for the possible modes you can choose from. Anything with {} braces indicates a parameter you can change.

 - <span style="color:orange">("SLEEP", 915, 13, 125, "OFF", 12, 6, { Payload length ranging from 0-232 })</span>
 - <span style="color:orange">("STANDBY", 915, 13, 125, "OFF", 12, 6, { Payload length ranging from 0-232 })</span>
 - <span style="color:orange">("IDLE", 915, 13, 125, "OFF", 12, 6, { Payload length ranging from 0-232 })</span>
 - <span style="color:orange">("TX", 915, { Output Power }, 125, "OFF", 12, 6, { Payload length ranging from 0-232 })</span>
 - <span style="color:orange">("RXCONTINUOUS", 915, 13, { Bandwidth }, { LNA Boost }, 12, 6, { Payload length ranging from 0-232 })</span>
 - <span style="color:orange">("RXSINGLE", 915, 13, { Bandwidth }, { LNA Boost }, 12, 6, { Payload length ranging from 0-232 })</span>
 - <span style="color:orange">("FSTX", 915, 13, 125, "OFF", 12, 6, { Payload length ranging from 0-232 })</span>
 - <span style="color:orange">("FSRX", 915, 13, 125, "OFF", 12, 6, { Payload length ranging from 0-232 })</span>
 - <span style="color:orange">("CAD", 915, 13, { Bandwidth }, "OFF", 12, 6, { Payload length ranging from 0-232 })</span>

You should also choose how long the sensor will spend in that configuration (duration, in seconds) and what is the transmission/ reception rate for SX1272
- <span style="color:orange">**Configuration duration (s)**</span>: {n} 
- <span style="color:orange">**Estimated Transmission - Reception Rate (s)**</span>: {n}
    - The time at which data is transmitted and received in a communication system. Realistically, this will be variable and dependent on many factors, including the amount of data that needs to be transmitted, if packets are expected at all and how much data is expected to be received. 
    - The tranmission and reception rate can not be smaller than the standard rate, which is given by -
        - Standard rate = (Payload Length (in bytes) * 8)/4800 

<span style="color:#18BF7D">For the following code, replace 'None' with appropriate values based on what you have learned above. Any lines without a 'None' should not be modified. Changing the values of the variables allows you to adjust a configuration consisting of a collection of these variables. At the end of the code, these configurations, in addition to their durations and the sampling rates, are put together in a list.</span>

In [ ]:
import micropip
await micropip.install("matplotlib")
import matplotlib.pyplot as plt
await micropip.install("numpy")
import numpy as np
from source.SX1272 import SX1272

# Example parameters

mode = "TX"
frequency = 915
output_power = 17
bandwidth = 125
lna_boost = "OFF"
spreading_factor = 12
coding_rate = 6
payload_size = 16

example_configuration = (mode, frequency, output_power, bandwidth, lna_boost, spreading_factor, coding_rate, payload_size) # Parentheses are used to group variables together in a 'tuple', a data type in Python 
duration = 30
transmission_reception_rate = 1


# Try creating 3 distinct configurations below by replacing None with appropriate values

# Configuration 1

mode = None
frequency = None
output_power = None
bandwidth = None
lna_boost = None
spreading_factor = 12
coding_rate = 6
payload_size = None

configuration_1 = (mode, frequency, output_power, bandwidth, lna_boost, spreading_factor, coding_rate, payload_size)
duration_1 = None
transmission_reception_rate_1 = None

# Configuration 2

mode = None
frequency = None
output_power = None
bandwidth = None
lna_boost = None
spreading_factor = 12
coding_rate = 6
payload_size = None

configuration_2 = (mode, frequency, output_power, bandwidth, lna_boost, spreading_factor, coding_rate, payload_size)
duration_2 = None
transmission_reception_rate_2 = None


# Configuration 3

mode = None
frequency = None
output_power = None
bandwidth = None
lna_boost = None
spreading_factor = 12
coding_rate = 6
payload_size = None

configuration_3 = (mode, frequency, output_power, bandwidth, lna_boost, spreading_factor, coding_rate, payload_size)
duration_3 = None
transmission_reception_rate_3 = None

# Below, we group together the configuration tuple with a duration and sampling rate in another tuple
# These tuples are added to a list called modes_ACC
modes_SX1 = [(configuration_1, duration_1, transmission_reception_rate_1), (configuration_2, duration_2, transmission_reception_rate_2), (configuration_3, duration_3, transmission_reception_rate_3)]

## <span style="color:tomato"> Configurations For A Lunar Mission </span>
Now that you have explored different configurations, let's consider choosing the ideal mode for a lunar mission. The modes we are particularly interested in are <span style="color:orange">SLEEP</span>, <span style="color:orange">STANDBY</span>, <span style="color:orange">TX</span>, <span style="color:orange">RXCONTINUOUS</span>, and <span style="color:orange">RXSINGLE</span>. <span style="color:orange">SLEEP</span> will be the primary mode for when data collection is occuring and we want the RF system to effectively be off. <span style="color:orange">STANDBY</span> will help to reduce power usage when transmissions are not actively occuring. <span style="color:orange">RXSINGLE</span> may also help to reduce power usage in cases where the LunaSat is not expecting data spread across multiple packets. The RX Continuous mode is not appropriate for the mission because we only need to recieve data samples only when needed. In other words, using RX Continuous wastes power. So we should only consider using the RX Single mode, TX mode, Sleep mode, Idle mode or the Standby mode.


<span style="color:#18BF7D">With these limitations in mind, revisit the configurations you set above and revise them so that they are well-suited for a lunar mission. </span>

## <span style="color:tomato"> Calculating Power Usage </span>

We can calculate the power usage based on the configuration to get an idea of how it's affected by the various settings. Power is measured in milliWatts, where 1 milliWatt is equivalent to 0.001 Joules of energy per second. 


<span style="color:#18BF7D">Do not modify the following code. Simply run the cell to see the output.</span>

In [ ]:
power_1 = SX1272.compute_power(SX1272,*configuration_1, transmission_reception_rate_1)
power_2 = SX1272.compute_power(SX1272,*configuration_2, transmission_reception_rate_2)
power_3 = SX1272.compute_power(SX1272,*configuration_3, transmission_recption_rate_3)
print("Config. 1: ", power_1, "milliWatts")
print("Config. 2: ", power_2, "milliWatts")
print("Config. 3: ", power_3, "milliWatts")

plt.bar(1, power_1, label='Config. 1')
plt.bar(2, power_2, label='Config. 2')
plt.bar(3, power_3, label='Config. 3')
plt.xticks([])
plt.ylabel('Power in mW')
plt.grid()
plt.legend()
plt.show()

## <span style="color:tomato"> Calculating Data Usage </span>

We can calculate the data usage for any configuration to get an idea of how it's affected by the various settings. It's important to note that we are concerned with the data usage based on the transmission or reception rate.

<span style="color:#18BF7D">Do not modify the following code. Simply run the cell to see the output.</span>

In [ ]:
data_1 = SX1272.compute_data(SX1272,*configuration_1, transmission_reception_rate_1)
data_2 = SX1272.compute_data(SX1272,*configuration_2, transmission_reception_rate_2)
data_3 = SX1272.compute_data(SX1272,*configuration_3, transmission_recption_rate_3)

print("Config. 1: ", data_1, "bytes per second")
print("Config. 2: ", data_2, "bytes per second")
print("Config. 3: ", data_3, "bytes per second")

plt.bar(1, data_1, label='Config. 1')
plt.bar(2, data_2, label='Config. 2')
plt.bar(3, data_3, label='Config. 3')
plt.xticks([])
plt.ylabel('Bytes/s')
plt.grid()
plt.legend()
plt.show()

## <span style="color:tomato"> Putting It All Together </span>

Finally, we'll take a look at the power and data usage of all of our configurations together. The time_step is the distance between data values in the plot, and the total duration is how long the sensor is active for in the model.

<span style="color:#18BF7D">Do not modify the following code. Simply run the cell to see the output.</span>

In [ ]:
time_step = 1
total_duration = duration_1 + duration_2 + duration_3 # total time to loop through configurations. Change if desired
rf = SX1272(time_step, total_duration, modes_SX1, loop_rate=20)
rf_power, rf_data, rf_time = rf.run_sim()


In [ ]:
%store rf_power
%store rf_data
%store rf_time

[Click here to continue to the next notebook.](2.0%20Combined%20PDM.ipynb)

If you would like to look at more detailed information for the SX1272 sensor, consider looking at the [SX1272 Datasheet](https://www.rfsolutions.co.uk/downloads/1462271988SX1272_SEMTECH.pdf).